## Get observations from program ID
Inspired by the [JWebbinar session about MAST](https://github.com/spacetelescope/jwebbinar_prep/blob/main/mast_session/Crowded_Field/Crowded_Field.ipynb).

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astroquery.mast import Observations
import numpy as np

Select the proposal ID, instrument, and some useful keywords (filters in this case).

In [ ]:
obs_table = Observations.query_criteria(obs_collection=["JWST"], 
                                        instrument_name=["NIRISS/IMAGE", "NIRISS/WFSS"],
                                        provenance_name=["CALJWST"], # Executed observations
                                        filters=['F115W','F150W','F200W'],
                                        proposal_id=[2079],
                                       )
obs_table

I could list all the products, but if there are too many it could time out. Let us devide the observations in baches and then filter and download one batch at a time.

In [ ]:
batch_size = 5
# Let's split up ``obs_table`` into batches according to our batch size.
obs_batches = [obs_table[i:i+batch_size] for i in range(0, len(obs_table), batch_size)]
single_group = obs_batches[0] # Useful to inspect the files obtained from one group
print("How many batches?", len(obs_batches))

Select the type of products needed. I do one at a time. I need:
- rate images
    - productType=["SCIENCE"]
    - productSubGroupDescription=['RATE']
    - calib_level=[2]
- level 2 associations for both spectroscopy and imaging
    - productType=["INFO"]
    - productSubGroupDescription=['ASN']
    - calib_level=[2]
- level 3 associations for imaging
    - productType=["INFO"]
    - productSubGroupDescription=['ASN']
    - dataproduct_type=["image"]
    - calib_level=[3]

In [ ]:
# Now let's get the products for each batch of observations, and filter down to only the products of interest.
for index, batch in enumerate(obs_batches):
#for index, batch in enumerate(single_group): # Useful to inspect the files obtained from one batch
    
    # Progress indicator...
    print('\n'+f'Batch #{index+1}')
    
    # Make a list of the `obsid` identifiers from our Astropy table of observations to get products for.
    obsids = batch['obsid']
    print('Working with the following ``obsid``s:')
    print(list(obsids))
    
    # Get list of products 
    products = Observations.get_product_list(obsids)
    
    # Filter the products to only get science products of calibration level 3 
    filtered_products = Observations.filter_products(products, 
                                                     productType=["INFO"],
                                                     productSubGroupDescription=['ASN'],
                                                     dataproduct_type=["image"],
                                                     calib_level=[3]
                                                    )
    #filtered_products
    # Download products for these records.
    print('Products downloaded:')
    print(filtered_products['productFilename'])
    #print(filtered_products)
    manifest = Observations.download_products(filtered_products)